In [1]:
from dotenv import load_dotenv
import os
import json
import pathlib
import pandas as pd
import datetime
import sqlite3
import sys
import yfinance as yf

from polygon import RESTClient

sys.path.append("../src")
from data import CRUD

load_dotenv()
%load_ext dotenv
%dotenv

In [2]:
INTERIM_DATA_FOLDER_PATH = pathlib.Path("../data/interim")

In [3]:
with open(INTERIM_DATA_FOLDER_PATH.joinpath('reference_store.json'), "r") as f:
    reference_store = json.load(f)

In [4]:
reference_store.keys()

dict_keys(['VIS', 'VFH', 'VDE', 'VGT', 'VHT'])

In [5]:
print(reference_store['VIS'].keys())
print(reference_store['VFH'].keys())
print(reference_store['VDE'].keys())
print(reference_store['VGT'].keys())
print(reference_store['VHT'].keys())

dict_keys(['CAT', 'UNP', 'GE', 'HON', 'RTX', 'UBER', 'DE', 'UPS', 'ADP', 'ETN'])
dict_keys(['JPM', 'BRK.A', 'V', 'MA', 'BAC', 'WFC', 'SPGI', 'GS', 'AXP', 'BLK'])
dict_keys(['XOM', 'CVX', 'COP', 'SLB', 'EOG', 'MPC', 'PSX', 'PSD', 'VLO', 'WMB'])
dict_keys(['MSFT', 'AAPL', 'NVDA', 'AVGO', 'ADBE', 'CRM', 'AMD', 'ACN', 'CSCO', 'ORCL'])
dict_keys(['LLY', 'UNH', 'MRK', 'ABBV', 'TMO', 'ABT', 'DHR', 'AMGN', 'PFE', 'ISRG'])


In [ ]:
# reference_store['VIS'].update({'UBER': {}, 'DE': {}, 'UPS': {}, 'ADP': {}, 'ETN': {}})
# reference_store['VFH'].update({'WFC': {}, 'SPGI': {}, 'GS': {}, 'AXP': {}, 'BLK': {}})
# reference_store['VDE'].update({'MPC': {}, 'PSX': {}, 'PSD': {}, 'VLO': {}, 'WMB': {}})
# reference_store['VGT'].update({'CRM': {}, 'AMD': {}, 'ACN': {}, 'CSCO': {}, 'ORCL': {}})
# reference_store['VHT'].update({'ABT': {}, 'DHR': {}, 'AMGN': {}, 'PFE': {}, 'ISRG': {}})

In [ ]:
reference_store['VIS'].keys()

In [ ]:
with open(INTERIM_DATA_FOLDER_PATH.joinpath('reference_store.json'), "w") as f:
    json.dump(reference_store, f)

# Yahoo finance API

In [ ]:
with sqlite3.connect("../src/data/finance_advisor_database.db") as conn:
    cur = conn.cursor()
    rest = cur.execute("select distinct etf_symbol, company_symbol from news;")

In [ ]:
ticker = yf.Tickers('VIS CAT')

In [ ]:
hist = ticker.history(start='2020-06-01', group_by='ticker')

In [ ]:
hist

# Check reference store

In [ ]:
INTERIM_DATA_FOLDER_PATH = pathlib.Path("../data/interim")
RAW_DATA_FOLDER_PATH = pathlib.Path("../data/raw")

In [ ]:
with open(INTERIM_DATA_FOLDER_PATH.joinpath("reference_store.json"), 'r') as f:
    reference_store = json.load(f)

# Polygon

In [ ]:
POLYGON_API_KEY = os.environ["POLYGON_API_KEY"]

In [ ]:
client = RESTClient(api_key=POLYGON_API_KEY)

In [ ]:
response = client.list_ticker_news(ticker='C', published_utc='2024-02-01')
aber = [a for a in response]

In [ ]:
aber

In [ ]:
for obj in aber:
    print(obj.title)